<a href="https://colab.research.google.com/github/dolphin1404/AI_lab/blob/main/data_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 도서-스크립트 변환 프로젝트: 데이터 수집 및 전처리

## 프로젝트 개요
- **목표**: 도서 텍스트를 비디오 스크립트 형식으로 변환하는 LLM 모델 개발
- **단계**: 데이터 수집 → 전처리 → 모델링 → 성능평가
- **일정**: 데이터 전처리 마감 - 10월 27일

## 데이터 전처리 목표
1. 공개 도서 아카이브에서 텍스트 데이터 수집 (Project Gutenberg, 국내 디지털 도서관)
2. 노이즈 제거 및 텍스트 정제
3. 핵심 요소 추출 (인물, 장소, 시간)
4. 도서 문체 → 스크립트 문체 변환을 위한 학습 데이터셋 구축

## 1. 환경 설정 및 라이브러리 설치

In [ ]:
# 필요한 라이브러리 설치
!pip install gutenberg requests beautifulsoup4 nltk spacy transformers datasets
!python -m spacy download en_core_web_sm
!python -m spacy download ko_core_news_sm

  Using cached Gutenberg-0.8.2-py3-none-any.whl.metadata (9.4 kB)
  Using cached SPARQLWrapper-2.0.0-py3-none-any.whl.metadata (2.0 kB)
  Using cached bsddb3-6.2.9.tar.gz (230 kB)
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 145.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencie

In [ ]:
# 라이브러리 임포트
import os
import re
import requests
import nltk
import spacy
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from collections import defaultdict
import json

# NLTK 데이터 다운로드
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')

# SpaCy 모델 로드
nlp_en = spacy.load('en_core_web_sm')

print("✓ All libraries imported successfully!")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


✓ All libraries imported successfully!


## 2. 데이터 수집 (Data Collection)

### 2.1 Project Gutenberg에서 도서 다운로드

In [ ]:
# 구텐버그에서 수집할 도서 ID 리스트 (랜덤 10개)# 인기있고 챕터 구분이 명확한 고전 소설들을 선택# 각 ID는 Project Gutenberg의 책 번호입니다popular_book_ids = [    1342,  # Pride and Prejudice by Jane Austen    2701,  # Moby Dick by Herman Melville      84,    # Frankenstein by Mary Shelley    1661,  # The Adventures of Sherlock Holmes by Arthur Conan Doyle    11,    # Alice's Adventures in Wonderland by Lewis Carroll    98,    # A Tale of Two Cities by Charles Dickens    74,    # The Adventures of Tom Sawyer by Mark Twain    345,   # Dracula by Bram Stoker    46,    # A Christmas Carol by Charles Dickens    1952,  # The Yellow Wallpaper by Charlotte Perkins Gilman]# 랜덤하게 10개 선택 (이미 10개이므로 전체 사용)random.seed(42)  # 재현성을 위한 시드 설정book_ids_to_process = popular_book_ids[:10]print(f"📚 총 {len(book_ids_to_process)}개의 도서를 처리합니다:")for book_id in book_ids_to_process:    print(f"  - Book ID: {book_id}")

✓ Gutenberg Collector initialized
Sample book IDs: [1342, 84, 98, 1661, 2701]


In [ ]:
# 구텐버그에서 수집할 도서 ID 리스트 (랜덤 10개)# 인기있고 챕터 구분이 명확한 고전 소설들을 선택# 각 ID는 Project Gutenberg의 책 번호입니다popular_book_ids = [    1342,  # Pride and Prejudice by Jane Austen    2701,  # Moby Dick by Herman Melville      84,    # Frankenstein by Mary Shelley    1661,  # The Adventures of Sherlock Holmes by Arthur Conan Doyle    11,    # Alice's Adventures in Wonderland by Lewis Carroll    98,    # A Tale of Two Cities by Charles Dickens    74,    # The Adventures of Tom Sawyer by Mark Twain    345,   # Dracula by Bram Stoker    46,    # A Christmas Carol by Charles Dickens    1952,  # The Yellow Wallpaper by Charlotte Perkins Gilman]# 랜덤하게 10개 선택 (이미 10개이므로 전체 사용)random.seed(42)  # 재현성을 위한 시드 설정book_ids_to_process = popular_book_ids[:10]print(f"📚 총 {len(book_ids_to_process)}개의 도서를 처리합니다:")for book_id in book_ids_to_process:    print(f"  - Book ID: {book_id}")

✓ Successfully downloaded book 1342
Book length: 743383 characters

First 500 characters:
*** START OF THE PROJECT GUTENBERG EBOOK 1342 ***




                            [Illustration:

                             GEORGE ALLEN
                               PUBLISHER

                        156 CHARING CROSS ROAD
                                LONDON

                             RUSKIN HOUSE
                                   ]

                            [Illustration:

               _Reading Jane’s Letters._      _Chap 34._
                               


## 3. 데이터 전처리 (Data Preprocessing)

### 3.1 텍스트 정제 (Text Cleaning)

In [ ]:
import re
import time

# nlp_en is not defined in the provided code, so I'm commenting it out.
# You would typically initialize a spaCy model here, for example:
# import spacy
# nlp_en = spacy.load("en_core_web_sm")

class TextPreprocessor:
    """텍스트 전처리를 위한 클래스"""

    def __init__(self):
        # self.nlp = nlp_en
        pass

    def remove_gutenberg_header_footer(self, text):
        """
        Project Gutenberg 헤더와 푸터 제거

        Args:
            text (str): 원본 텍스트

        Returns:
            str: 헤더/푸터가 제거된 텍스트
        """
        # 시작 마커 찾기
        start_markers = [
            "*** START OF THIS PROJECT GUTENBERG",
            "*** START OF THE PROJECT GUTENBERG",
            "***START OF THE PROJECT GUTENBERG"
        ]

        # 종료 마커 찾기
        end_markers = [
            "*** END OF THIS PROJECT GUTENBERG",
            "*** END OF THE PROJECT GUTENBERG",
            "***END OF THE PROJECT GUTENBERG"
        ]

        start_idx = 0
        for marker in start_markers:
            idx = text.find(marker)
            if idx != -1:
                start_idx = text.find('\n', idx) + 1
                break

        end_idx = len(text)
        for marker in end_markers:
            idx = text.find(marker)
            if idx != -1:
                end_idx = idx
                break

        return text[start_idx:end_idx].strip()

    def remove_table_of_contents(self, text):
        """
        목차(Table of Contents) 섹션 제거 (v4 - 개선된 정확도)

        개선 사항:
        - 실제 챕터와 목차 항목 구분 강화
        - Chapter I부터 정확히 보존
        - 목차 패턴 더 정밀하게 감지

        Args:
            text (str): 원본 텍스트

        Returns:
            str: 목차가 제거된 텍스트
        """
        lines = text.split('\n')
        result_lines = []
        in_toc = False
        toc_start_idx = -1
        toc_line_count = 0
        consecutive_heading_lines = 0
        found_first_chapter = False

        for i, line in enumerate(lines):
            line_stripped = line.strip()
            line_lower = line_stripped.lower()

            # 목차 시작 감지 (더 정확한 패턴)
            if not in_toc and not found_first_chapter:
                # "CONTENTS" 단독으로 나오는 경우 (일반적인 목차 시작)
                if line_stripped.upper() in ['CONTENTS', 'TABLE OF CONTENTS', 'LIST OF CHAPTERS']:
                    in_toc = True
                    toc_start_idx = i
                    toc_line_count = 0
                    continue

                # "Heading to Chapter" 패턴 여러 줄 연속 (Pride and Prejudice 스타일)
                if 'heading to chapter' in line_lower or 'heading to CHAPTER' in line_lower:
                    consecutive_heading_lines += 1
                    # 2줄 이상 연속으로 나오면 목차
                    if consecutive_heading_lines >= 2:
                        in_toc = True
                        toc_start_idx = i - consecutive_heading_lines
                        continue
                else:
                    if 0 < consecutive_heading_lines < 2:
                        # 1줄만 있었으면 실제 내용일 수 있음
                        consecutive_heading_lines = 0

            # 실제 챕터 시작 확인 (목차 종료 또는 첫 챕터 발견)
            # 더 엄격한 조건: 줄이 짧고(80자 이하), 패턴이 명확하고, 다음 줄에 내용이 있어야 함
            is_real_chapter = False
            if len(line_stripped) <= 80:
                # "CHAPTER I", "Chapter 1", "CHAPTER ONE" 등의 패턴
                chapter_pattern = r'^\s*(CHAPTER|Chapter)\s+(I|II|III|IV|V|VI|VII|VIII|IX|X|XI|XII|XIII|XIV|XV|XVI|XVII|XVIII|XIX|XX|XXI|XXII|XXIII|XXIV|XXV|XXVI|XXVII|XXVIII|XXIX|XXX|XXXI|XXXII|XXXIII|XXXIV|XXXV|XXXVI|XXXVII|XXXVIII|XXXIX|XL|XLI|XLII|XLIII|XLIV|XLV|XLVI|XLVII|XLVIII|XLIX|L|LI|LII|LIII|LIV|LV|LVI|LVII|LVIII|LIX|LX|LXI|LXII|LXIII|LXIV|LXV|LXVI|LXVII|LXVIII|LXIX|LXX|\d+|One|Two|Three|Four|Five|Six|Seven|Eight|Nine|Ten|Eleven|Twelve|Thirteen|Fourteen|Fifteen|Sixteen|Seventeen|Eighteen|Nineteen|Twenty)\.?\s*$'
                chapter_match = re.match(chapter_pattern, line_stripped)

                if chapter_match:
                    # 다음 몇 줄을 확인하여 실제 챕터 내용이 있는지 검증
                    has_content_after = False
                    for j in range(i + 1, min(i + 10, len(lines))):
                        next_line = lines[j].strip()
                        # 빈 줄이 아니고, 40자 이상의 실제 문장이 있으면 챕터로 간주
                        if next_line and len(next_line) > 40:
                            # 목차 키워드가 없어야 함
                            if 'heading to' not in next_line.lower() and 'page' not in next_line.lower():
                                has_content_after = True
                                break

                    if has_content_after:
                        is_real_chapter = True
                        found_first_chapter = True

                        # 목차 중이었다면 목차 종료
                        if in_toc and toc_line_count > 3:
                            in_toc = False
                            # 목차 부분 스킵 (이미 추가 안 함)

            # 목차 중이면 줄 카운트만 증가하고 추가 안 함
            if in_toc:
                toc_line_count += 1
                # 목차가 너무 길면 (150줄 이상) 종료
                if toc_line_count > 150:
                    in_toc = False
                continue # 목차 내용은 결과에 추가하지 않음

            # 실제 챕터를 만났거나, 목차 상태가 아니면 라인 추가
            if is_real_chapter or not in_toc:
                result_lines.append(line)

        return '\n'.join(result_lines)


    def clean_text(self, text):
        """
        기본 텍스트 정제
        - 과도한 공백 제거
        - 특수 문자 정규화
        - 줄바꿈 정규화

        Args:
            text (str): 원본 텍스트

        Returns:
            str: 정제된 텍스트
        """
        # 여러 줄바꿈을 단일 줄바꿈으로
        text = re.sub(r'\n\s*\n', '\n\n', text)
        # 여러 공백을 단일 공백으로
        text = re.sub(r' +', ' ', text)
        # 줄 시작/끝 공백 제거
        text = '\n'.join(line.strip() for line in text.split('\n'))
        return text.strip()

    def split_into_chapters(self, text):
        """
        텍스트를 챕터별로 분할 (개선된 버전 v2)

        개선 사항:
        - 목차(Table of Contents) 자동 제거
        - "Heading to Chapter" 같은 목차 항목 필터링
        - 실제 챕터 내용만 추출
        - 더 정확한 챕터 감지

        지원하는 형식:
        - "CHAPTER I", "CHAPTER 1", "CHAPTER ONE"
        - "Chapter I.", "Chapter 1.", "Chapter One."
        - 실제 챕터 제목이 있는 경우 (e.g., "CHAPTER I. The Beginning")

        Args:
            text (str): 전체 텍스트

        Returns:
            list: 챕터별 텍스트 리스트
        """
        # 1단계: 목차 제거
        text_without_toc = self.remove_table_of_contents(text)

        # 2단계: 챕터 패턴 매칭
        # 안전한 패턴 사용 (catastrophic backtracking 방지)
        word_numbers = "One|Two|Three|Four|Five|Six|Seven|Eight|Nine|Ten|Eleven|Twelve|Thirteen|Fourteen|Fifteen|Sixteen|Seventeen|Eighteen|Nineteen|Twenty|Twenty-one|Twenty-two|Twenty-three|Twenty-four|Twenty-five|Twenty-six|Twenty-seven|Twenty-eight|Twenty-nine|Thirty|Thirty-one|Thirty-two|Thirty-three|Thirty-four|Thirty-five|Thirty-six|Thirty-seven|Thirty-eight|Thirty-nine|Forty|Forty-one|Forty-two|Forty-three|Forty-four|Forty-five|Forty-six|Forty-seven|Forty-eight|Forty-nine|Fifty|Fifty-one|Fifty-two|Fifty-three|Fifty-four|Fifty-five|Fifty-six|Fifty-seven|Fifty-eight|Fifty-nine|Sixty|Sixty-one"

        patterns = [
            r'\n\s*(CHAPTER|Chapter)\s+([IVXLCDM]+|\d+|' + word_numbers + r')(?:\.\s*|\s+|\n)',
            # Pattern 2: "BOOK I", "PART I"
            r'\n\s*(BOOK|Book|PART|Part)\s+([IVXLCDM]+|\d+)(?:\.\s*|\s+|\n)',
        ]

        best_split = None
        best_count = 0
        best_pattern_idx = -1

        # 각 패턴 시도 (타임아웃 추가)
        for idx, pattern in enumerate(patterns):
            try:
                start_time = time.time()
                splits = re.split(pattern, text_without_toc)

                # 타임아웃 체크 (3초)
                if time.time() - start_time > 3:
                    print(f"Warning: Pattern {idx} took too long, skipping")
                    continue

                # 챕터 수 계산 (3개 요소가 1개 챕터: prefix, number, content)
                chapter_count = (len(splits) - 1) // 3 if len(splits) > 1 else 0

                # 합리적인 범위의 챕터 수 (2-150개)
                if 2 <= chapter_count <= 150:
                    # 챕터 평균 길이 확인 (너무 짧으면 목차일 가능성)
                    total_length = sum(len(splits[i]) for i in range(2, len(splits), 3) if i < len(splits))
                    avg_length = total_length / chapter_count if chapter_count > 0 else 0

                    # 평균 길이가 300자 이상이어야 실제 챕터
                    if avg_length >= 300 and chapter_count > best_count:
                        best_count = chapter_count
                        best_split = (pattern, splits)
                        best_pattern_idx = idx
            except Exception as e:
                print(f"Warning: Pattern {idx} failed with error: {e}")
                continue

        # 3단계: 챕터 추출
        if best_split:
            pattern, chapters = best_split
            result = []

            for i in range(1, len(chapters), 3):
                if i + 2 <= len(chapters):
                    chapter_prefix = chapters[i].strip()    # "CHAPTER" or "Chapter"
                    chapter_number = chapters[i + 1].strip()  # "I", "1", etc.
                    chapter_content = chapters[i + 2].strip() if i + 2 < len(chapters) else ""

                    # 챕터 내용이 충분히 긴지 확인 (최소 200자)
                    if len(chapter_content) < 200:
                        continue

                    # 챕터 제목에서 실제 제목 추출
                    content_lines = chapter_content.split('\n', 2)
                    chapter_title_suffix = ""

                    # 첫 줄이 짧으면 (60자 이하) 챕터 제목으로 간주
                    if content_lines and len(content_lines[0]) <= 60 and content_lines[0].strip():
                        chapter_title_suffix = content_lines[0].strip()
                        # 제목을 제외한 나머지가 내용
                        if len(content_lines) > 1:
                            chapter_content = '\n'.join(content_lines[1:]).strip()

                    # 최종 챕터 제목 생성
                    if chapter_prefix:
                        if chapter_title_suffix:
                            chapter_title = f"{chapter_prefix} {chapter_number}. {chapter_title_suffix}"
                        else:
                            chapter_title = f"{chapter_prefix} {chapter_number}"
                    else:
                        chapter_title = f"Chapter {chapter_number}"

                    if chapter_content:  # 내용이 있을 때만 추가
                        result.append({
                            'title': chapter_title,
                            'content': chapter_content
                        })

            # 결과 검증: 최소 2개 이상의 챕터
            if len(result) >= 2:
                return result

        # 4단계: Fallback - 수동 라인 분석
        return self._fallback_chapter_split(text_without_toc)


    def _fallback_chapter_split(self, text):
        """
        Fallback 방법: 줄 단위로 챕터 마커 찾기
        목차 항목은 제외하고 실제 챕터만 추출
        """
        lines = text.split('\n')
        chapters = []
        current_chapter = None
        current_content = []

        for i, line in enumerate(lines):
            line_stripped = line.strip()
            line_upper = line_stripped.upper()

            # 목차 항목 건너뛰기
            if 'HEADING TO' in line_upper or 'CONTENTS' in line_upper:
                continue

            # 챕터 헤더 감지
            is_chapter = False
            chapter_title = None

            # 짧은 줄 (60자 이하)에서만 챕터 검사
            if len(line_stripped) <= 60 and line_stripped:
                # "CHAPTER X" 형식
                chapter_match = re.match(r'^(CHAPTER|Chapter)\s+([IVXLCDM]+|\d+)', line_stripped)
                if chapter_match:
                    is_chapter = True
                    chapter_title = line_stripped
                # "BOOK X", "PART X" 형식
                elif re.match(r'^(BOOK|Book|PART|Part)\s+([IVXLCDM]+|\d+)', line_stripped):
                    is_chapter = True
                    chapter_title = line_stripped

            if is_chapter and chapter_title:
                # 이전 챕터 저장
                if current_chapter and current_content:
                    content_text = '\n'.join(current_content).strip()
                    # 내용이 충분히 긴 경우만 (최소 300자)
                    if len(content_text) >= 300:
                        chapters.append({
                            'title': current_chapter,
                            'content': content_text
                        })

                # 새 챕터 시작
                current_chapter = chapter_title
                current_content = []
            else:
                # 현재 챕터에 내용 추가
                if current_chapter:  # 챕터가 시작된 후에만
                    current_content.append(line)

        # 마지막 챕터 저장
        if current_chapter and current_content:
            content_text = '\n'.join(current_content).strip()
            if len(content_text) >= 300:
                chapters.append({
                    'title': current_chapter,
                    'content': content_text
                })

        # 최소 2개 이상의 챕터가 있어야 유효
        if len(chapters) >= 2:
            return chapters
        else:
            return [{'title': 'Full Text', 'content': text}]


In [ ]:
preprocessor = TextPreprocessor()
# 샘플 도서에 전처리 적용
if book_text:
    # 헤더/푸터 제거
    cleaned_text = preprocessor.remove_gutenberg_header_footer(book_text)
    cleaned_text = preprocessor.clean_text(cleaned_text)

    print(f"✓ Original length: {len(book_text)} characters")
    print(f"✓ Cleaned length: {len(cleaned_text)} characters")
    print(f"✓ Removed: {len(book_text) - len(cleaned_text)} characters")

    # 챕터 분할
    chapters = preprocessor.split_into_chapters(cleaned_text)
    print(f"\n✓ Found {len(chapters)} chapters")

    if chapters:
        print(f"\nFirst chapter: {chapters[0]['title']}")
        print(f"Content preview: {chapters[0]['content'][:300]}...")

✓ Original length: 743383 characters
✓ Cleaned length: 720973 characters
✓ Removed: 22410 characters

✓ Found 61 chapters

First chapter: Chapter I.]
Content preview: It is a truth universally acknowledged, that a single man in possession
of a good fortune must be in want of a wife.

However little known the feelings or views of such a man may be on his
first entering a neighbourhood, this truth is so well fixed in the minds
of the surrounding families, that he i...


### 3.2 핵심 요소 추출 (Entity Extraction)

인물, 장소, 시간 등 핵심 요소를 추출하여 스크립트 변환에 활용

In [ ]:
class EntityExtractor:
    """Named Entity Recognition을 통한 핵심 요소 추출 클래스"""

    def __init__(self):
        self.nlp = nlp_en

    def extract_entities(self, text, max_length=1000000):
        """
        텍스트에서 개체명 추출

        Args:
            text (str): 분석할 텍스트
            max_length (int): 처리할 최대 텍스트 길이

        Returns:
            dict: 카테고리별 개체명 사전
        """
        # SpaCy의 max_length 설정
        self.nlp.max_length = max_length

        # 텍스트 분석
        doc = self.nlp(text[:max_length])

        # 개체명 분류
        entities = {
            'PERSON': [],      # 인물
            'GPE': [],          # 지정학적 개체 (도시, 국가 등)
            'LOC': [],          # 위치
            'DATE': [],         # 날짜
            'TIME': [],         # 시간
            'ORG': [],          # 조직
            'EVENT': []         # 이벤트
        }

        for ent in doc.ents:
            if ent.label_ in entities:
                entities[ent.label_].append(ent.text)

        # 중복 제거 및 빈도수 계산
        for key in entities:
            entity_counts = defaultdict(int)
            for entity in entities[key]:
                entity_counts[entity] += 1
            entities[key] = sorted(entity_counts.items(), key=lambda x: x[1], reverse=True)

        return entities

    def get_main_characters(self, entities, top_n=10):
        """
        주요 인물 추출

        Args:
            entities (dict): extract_entities의 결과
            top_n (int): 반환할 주요 인물 수

        Returns:
            list: 주요 인물 리스트
        """
        return entities['PERSON'][:top_n]

    def get_main_locations(self, entities, top_n=10):
        """
        주요 장소 추출

        Args:
            entities (dict): extract_entities의 결과
            top_n (int): 반환할 주요 장소 수

        Returns:
            list: 주요 장소 리스트
        """
        locations = entities['GPE'] + entities['LOC']
        # 빈도수로 재정렬
        location_dict = defaultdict(int)
        for loc, count in locations:
            location_dict[loc] += count
        return sorted(location_dict.items(), key=lambda x: x[1], reverse=True)[:top_n]

# 사용 예시
extractor = EntityExtractor()
print("✓ Entity Extractor initialized")

✓ Entity Extractor initialized


In [ ]:
# 샘플 챕터에서 개체명 추출
if chapters:
    # 첫 번째 챕터 분석
    sample_chapter = chapters[0]['content']
    entities = extractor.extract_entities(sample_chapter)

    print("✓ Entity extraction completed\n")

    # 주요 인물
    main_characters = extractor.get_main_characters(entities, top_n=5)
    print("Main Characters:")
    for char, count in main_characters:
        print(f"  - {char}: {count} mentions")

    # 주요 장소
    main_locations = extractor.get_main_locations(entities, top_n=5)
    print("\nMain Locations:")
    for loc, count in main_locations:
        print(f"  - {loc}: {count} mentions")

    # 시간 정보
    if entities['DATE']:
        print("\nTemporal Information:")
        for date, count in entities['DATE'][:5]:
            print(f"  - {date}: {count} mentions")

✓ Entity extraction completed

Main Characters:
  - Bennet: 6 mentions
  - Bingley: 4 mentions
  - George Allen: 3 mentions
  - Lizzy: 3 mentions
  - Long: 2 mentions

Main Locations:
  - England: 1 mentions
  - Lydia: 1 mentions

Temporal Information:
  - 1894: 3 mentions
  - one day: 1 mentions
  - Monday: 1 mentions
  - the end of next week: 1 mentions
  - five thousand a year: 1 mentions


### 3.3 스크립트 변환을 위한 데이터 구조화

도서 텍스트를 스크립트 학습에 적합한 형태로 변환

In [ ]:
class ScriptFormatter:
    """도서 텍스트를 스크립트 형식으로 변환하는 클래스"""

    def __init__(self):
        self.nlp = nlp_en

    def extract_dialogues(self, text):
        """
        텍스트에서 대화문 추출

        Args:
            text (str): 분석할 텍스트

        Returns:
            list: 대화문 리스트
        """
        # 따옴표로 둘러싸인 대화문 추출
        dialogue_pattern = r'["\']([^"\']+)["\']'
        dialogues = re.findall(dialogue_pattern, text)

        # 짧은 대화 필터링 (3단어 이상)
        dialogues = [d for d in dialogues if len(d.split()) >= 3]

        return dialogues

    def extract_narrative(self, text):
        """
        서술 부분 추출 (대화가 아닌 부분)

        Args:
            text (str): 분석할 텍스트

        Returns:
            str: 서술 텍스트
        """
        # 대화문 제거
        narrative = re.sub(r'["\'][^"\']+["\']', '', text)

        # 정제
        narrative = re.sub(r' +', ' ', narrative)
        narrative = re.sub(r'\n\s*\n', '\n\n', narrative)

        return narrative.strip()

    def create_scene_structure(self, chapter_text, entities):
        """
        챕터를 씬 구조로 변환

        Args:
            chapter_text (str): 챕터 텍스트
            entities (dict): 추출된 개체명

        Returns:
            dict: 씬 구조 정보
        """
        # 문장 단위로 분할
        doc = self.nlp(chapter_text[:100000])  # 처리 속도를 위해 제한
        sentences = [sent.text for sent in doc.sents]

        # 대화문과 서술 분리
        dialogues = self.extract_dialogues(chapter_text)
        narrative = self.extract_narrative(chapter_text)

        scene_data = {
            'characters': [char for char, _ in entities.get('PERSON', [])[:5]],
            'locations': [loc for loc, _ in (entities.get('GPE', []) + entities.get('LOC', []))[:3]],
            'dialogues': dialogues[:10],
            'narrative_sentences': sentences[:20],
            'total_sentences': len(sentences),
            'total_dialogues': len(dialogues)
        }

        return scene_data

# 사용 예시
formatter = ScriptFormatter()
print("✓ Script Formatter initialized")

✓ Script Formatter initialized


In [ ]:
# 샘플 챕터를 씬 구조로 변환
if chapters and entities:
    scene_data = formatter.create_scene_structure(chapters[0]['content'], entities)

    print("✓ Scene structure created\n")
    print(f"Scene Information:")
    print(f"  - Main Characters: {', '.join(scene_data['characters'])}")
    print(f"  - Locations: {', '.join(scene_data['locations'])}")
    print(f"  - Total Sentences: {scene_data['total_sentences']}")
    print(f"  - Total Dialogues: {scene_data['total_dialogues']}")

    print("\nSample Dialogues:")
    for i, dialogue in enumerate(scene_data['dialogues'][:3], 1):
        print(f"  {i}. \"{dialogue}\"")

    print("\nSample Narrative:")
    for i, sentence in enumerate(scene_data['narrative_sentences'][:3], 1):
        print(f"  {i}. {sentence}")

✓ Scene structure created

Scene Information:
  - Main Characters: Bennet, Bingley, George Allen, Lizzy, Long
  - Locations: England, Lydia
  - Total Sentences: 53
  - Total Dialogues: 0

Sample Dialogues:

Sample Narrative:
  1. It is a truth universally acknowledged, that a single man in possession
of a good fortune must be in want of a wife.


  2. However little known the feelings or views of such a man may be on his
first entering a neighbourhood, this truth is so well fixed in the minds
of the surrounding families, that he is considered as the rightful
property of some one or other of their daughters.


  3. “My dear Mr. Bennet,” said his lady to him one day, “have you heard that
Netherfield Park is let at last?”

Mr. Bennet replied that he had not.




## 4. 학습 데이터셋 구축

### 4.1 전체 파이프라인 실행

In [ ]:
class BookToScriptPipeline:
    """도서에서 스크립트 학습 데이터까지의 전체 파이프라인"""

    def __init__(self):
        self.collector = GutenbergCollector()
        self.preprocessor = TextPreprocessor()
        self.extractor = EntityExtractor()
        self.formatter = ScriptFormatter()

    def process_book(self, book_id):
        """
        단일 도서 처리

        Args:
            book_id (int): 도서 ID

        Returns:
            dict: 처리된 데이터
        """
        print(f"\nProcessing book {book_id}...")

        # 1. 도서 다운로드
        book_text = self.collector.download_book(book_id)
        if not book_text:
            return None
        print(f"  ✓ Downloaded ({len(book_text)} chars)")

        # 2. 전처리
        cleaned_text = self.preprocessor.remove_gutenberg_header_footer(book_text)
        cleaned_text = self.preprocessor.clean_text(cleaned_text)
        print(f"  ✓ Cleaned ({len(cleaned_text)} chars)")

        # 3. 챕터 분할
        chapters = self.preprocessor.split_into_chapters(cleaned_text)
        print(f"  ✓ Split into {len(chapters)} chapters")

        # 4. 각 챕터별 처리
        processed_chapters = []
        for i, chapter in enumerate(chapters[:5]):  # 처음 5개 챕터만 처리 (예시)
            # 개체명 추출
            entities = self.extractor.extract_entities(chapter['content'])

            # 씬 구조 생성
            scene_data = self.formatter.create_scene_structure(chapter['content'], entities)

            processed_chapters.append({
                'chapter_title': chapter['title'],
                'chapter_number': i + 1,
                'original_text': chapter['content'],
                'entities': entities,
                'scene_data': scene_data
            })

        print(f"  ✓ Processed {len(processed_chapters)} chapters")

        return {
            'book_id': book_id,
            'total_length': len(cleaned_text),
            'total_chapters': len(chapters),
            'processed_chapters': processed_chapters
        }

    def process_multiple_books(self, book_ids, output_dir='./processed_data'):
        """
        여러 도서 처리 및 저장

        Args:
            book_ids (list): 도서 ID 리스트
            output_dir (str): 출력 디렉토리

        Returns:
            list: 처리 결과 리스트
        """
        os.makedirs(output_dir, exist_ok=True)
        results = []

        for book_id in book_ids:
            result = self.process_book(book_id)
            if result:
                results.append(result)

                # JSON으로 저장
                output_file = os.path.join(output_dir, f'book_{book_id}.json')
                with open(output_file, 'w', encoding='utf-8') as f:
                    json.dump(result, f, ensure_ascii=False, indent=2)
                print(f"  ✓ Saved to {output_file}")

        print(f"\n✓ Completed processing {len(results)} books")
        return results

# 파이프라인 초기화
pipeline = BookToScriptPipeline()
print("✓ Pipeline initialized")

✓ Pipeline initialized


In [ ]:
# 구텐버그에서 수집할 도서 ID 리스트 (랜덤 10개)# 인기있고 챕터 구분이 명확한 고전 소설들을 선택# 각 ID는 Project Gutenberg의 책 번호입니다popular_book_ids = [    1342,  # Pride and Prejudice by Jane Austen    2701,  # Moby Dick by Herman Melville      84,    # Frankenstein by Mary Shelley    1661,  # The Adventures of Sherlock Holmes by Arthur Conan Doyle    11,    # Alice's Adventures in Wonderland by Lewis Carroll    98,    # A Tale of Two Cities by Charles Dickens    74,    # The Adventures of Tom Sawyer by Mark Twain    345,   # Dracula by Bram Stoker    46,    # A Christmas Carol by Charles Dickens    1952,  # The Yellow Wallpaper by Charlotte Perkins Gilman]# 랜덤하게 10개 선택 (이미 10개이므로 전체 사용)random.seed(42)  # 재현성을 위한 시드 설정book_ids_to_process = popular_book_ids[:10]print(f"📚 총 {len(book_ids_to_process)}개의 도서를 처리합니다:")for book_id in book_ids_to_process:    print(f"  - Book ID: {book_id}")


Processing book 1342...
  ✓ Downloaded (743383 chars)
  ✓ Cleaned (720973 chars)
  ✓ Split into 61 chapters
  ✓ Processed 5 chapters
  ✓ Saved to ./processed_data/book_1342.json

✓ Completed processing 1 books

=== Processing Summary ===

Book ID: 1342
  Total Length: 720,973 characters
  Total Chapters: 61
  Processed Chapters: 5


## 5. 성능 평가 지표 준비

### 5.1 BLEU Score 구현

In [ ]:
from nltk.translate.bleu_score import sentence_bleu, corpus_bleu, SmoothingFunction

class EvaluationMetrics:
    """모델 성능 평가를 위한 메트릭 클래스"""

    def __init__(self):
        self.smooth = SmoothingFunction()

    def calculate_bleu(self, reference, candidate, weights=(0.25, 0.25, 0.25, 0.25)):
        """
        BLEU 점수 계산

        Args:
            reference (str): 참조 텍스트
            candidate (str): 생성된 텍스트
            weights (tuple): n-gram 가중치

        Returns:
            float: BLEU 점수
        """
        # 토큰화
        reference_tokens = reference.split()
        candidate_tokens = candidate.split()

        # BLEU 계산 (smoothing 적용)
        bleu_score = sentence_bleu(
            [reference_tokens],
            candidate_tokens,
            weights=weights,
            smoothing_function=self.smooth.method1
        )

        return bleu_score

    def calculate_bleu_variants(self, reference, candidate):
        """
        다양한 BLEU 변형 계산 (BLEU-1 ~ BLEU-4)

        Args:
            reference (str): 참조 텍스트
            candidate (str): 생성된 텍스트

        Returns:
            dict: BLEU 변형 점수
        """
        return {
            'BLEU-1': self.calculate_bleu(reference, candidate, weights=(1, 0, 0, 0)),
            'BLEU-2': self.calculate_bleu(reference, candidate, weights=(0.5, 0.5, 0, 0)),
            'BLEU-3': self.calculate_bleu(reference, candidate, weights=(0.33, 0.33, 0.33, 0)),
            'BLEU-4': self.calculate_bleu(reference, candidate, weights=(0.25, 0.25, 0.25, 0.25))
        }

# 평가 메트릭 초기화
metrics = EvaluationMetrics()
print("✓ Evaluation Metrics initialized")

# 테스트 예시
reference_text = "The quick brown fox jumps over the lazy dog"
candidate_text = "The quick brown fox jumps over a lazy dog"

bleu_scores = metrics.calculate_bleu_variants(reference_text, candidate_text)
print("\nBLEU Score Examples:")
for metric, score in bleu_scores.items():
    print(f"  {metric}: {score:.4f}")

✓ Evaluation Metrics initialized

BLEU Score Examples:
  BLEU-1: 0.8889
  BLEU-2: 0.8165
  BLEU-3: 0.7273
  BLEU-4: 0.6606


## 6. 다음 단계 및 계획

### 완료된 작업
- ✅ Project Gutenberg 데이터 수집 모듈
- ✅ 텍스트 전처리 (노이즈 제거, 정제)
- ✅ 챕터 분할 기능
- ✅ 개체명 추출 (인물, 장소, 시간)
- ✅ 스크립트 변환을 위한 데이터 구조화
- ✅ BLEU 평가 지표 구현

### 향후 작업 (10월 27일까지)
1. **데이터 수집 확대**
   - 더 많은 도서 다운로드 및 처리
   - 다양한 장르 확보 (소설, 드라마, 미스터리 등)
   - 국내 디지털 도서관 데이터 수집 방법 연구

2. **전처리 고도화**
   - 대화문과 서술 분리 정확도 향상
   - 장면 전환 감지 알고리즘 개발
   - 감정/톤 분석 추가

3. **데이터셋 품질 검증**
   - 결측치 및 이상치 검사
   - 데이터 통계 분석
   - 샘플 데이터 검토

### 모델링 준비 (10월 29일 이후)
- LLM 모델 선택 (GPT-2, T5, BART 등)
- Fine-tuning 전략 수립
- 학습 데이터 포맷 정의

### 성능 평가 계획
- BLEU Score: 텍스트 유사도 측정
- FVD (선택): 비디오 품질 평가 (여유가 있을 경우)
- CLIPScore (선택): 텍스트-이미지 유사도 (여유가 있을 경우)

## 7. 진행 상황 저장

In [ ]:
# 진행 상황 요약 저장
progress_summary = {
    'date': '2025-10-12',
    'phase': 'Data Preprocessing',
    'completed_tasks': [
        'Data collection module (Project Gutenberg)',
        'Text cleaning and preprocessing',
        'Chapter segmentation',
        'Entity extraction (characters, locations, time)',
        'Script formatting structure',
        'BLEU evaluation metric'
    ],
    'next_tasks': [
        'Expand dataset collection',
        'Enhance preprocessing accuracy',
        'Data quality validation',
        'Prepare for modeling phase'
    ],
    'deadline': '2025-10-27',
    'team_notes': 'Following professor\'s guidance - parallel work on dataset collection and framework setup'
}

# JSON으로 저장
with open('progress_summary.json', 'w', encoding='utf-8') as f:
    json.dump(progress_summary, f, ensure_ascii=False, indent=2)

print("✓ Progress summary saved to 'progress_summary.json'")
print("\nCurrent Status:")
print(f"  Phase: {progress_summary['phase']}")
print(f"  Deadline: {progress_summary['deadline']}")
print(f"  Completed: {len(progress_summary['completed_tasks'])} tasks")
print(f"  Remaining: {len(progress_summary['next_tasks'])} tasks")

✓ Progress summary saved to 'progress_summary.json'

Current Status:
  Phase: Data Preprocessing
  Deadline: 2025-10-27
  Completed: 6 tasks
  Remaining: 4 tasks


### 3.1.1 챕터 분할 예시 및 디버깅

개선된 챕터 분할 알고리즘 테스트

In [ ]:
# 챕터 분할 테스트 및 디버깅
if 'book_text' in dir() and 'cleaned_text' in dir():
    print("=== 챕터 분할 디버깅 ===")
    print(f"\n1. 원본 텍스트 샘플 (처음 1000자):")
    print(cleaned_text[:1000])
    print("\n" + "="*60 + "\n")

    # 챕터 패턴 수동 검색
    print("2. 챕터 패턴 검색 결과:")

    patterns_to_test = [
        ("CHAPTER/Chapter + 번호", r'\n\s*(CHAPTER|Chapter)\s+([IVXLCDM]+|\d+)'),
        ("로마자/숫자만", r'\n\s*([IVXLCDM]+|\d+)\.?\s*\n'),
        ("BOOK/PART + 번호", r'\n\s*(BOOK|Book|PART|Part)\s+([IVXLCDM]+|\d+)'),
    ]

    for pattern_name, pattern in patterns_to_test:
        matches = re.findall(pattern, cleaned_text)
        print(f"  - {pattern_name}: {len(matches)}개 발견")
        if matches and len(matches) <= 10:
            print(f"    예시: {matches[:5]}")

    print("\n" + "="*60 + "\n")

    # 실제 챕터 분할 수행
    print("3. 챕터 분할 결과:")
    chapters = preprocessor.split_into_chapters(cleaned_text)
    print(f"  총 {len(chapters)}개 챕터 발견\n")

    # 처음 3개 챕터 미리보기
    for i, chapter in enumerate(chapters[:3], 1):
        print(f"  Chapter {i}:")
        print(f"    제목: '{chapter['title']}'")
        print(f"    길이: {len(chapter['content'])} 문자")
        print(f"    내용 미리보기: {chapter['content'][:150]}...")
        print()

    # 챕터 길이 통계
    if len(chapters) > 1:
        chapter_lengths = [len(ch['content']) for ch in chapters]
        print(f"\n4. 챕터 길이 통계:")
        print(f"  - 평균: {sum(chapter_lengths)/len(chapter_lengths):.0f} 문자")
        print(f"  - 최소: {min(chapter_lengths)} 문자")
        print(f"  - 최대: {max(chapter_lengths)} 문자")

    print("\n" + "="*60)
    print("✓ 챕터 분할 분석 완료")
else:
    print("먼저 샘플 도서를 다운로드하고 정제하세요.")

=== 챕터 분할 디버깅 ===

1. 원본 텍스트 샘플 (처음 1000자):
[Illustration:

GEORGE ALLEN
PUBLISHER

156 CHARING CROSS ROAD
LONDON

RUSKIN HOUSE
]

[Illustration:

_Reading Jane’s Letters._ _Chap 34._
]

PRIDE.
and
PREJUDICE

by
Jane Austen,

with a Preface by
George Saintsbury
and
Illustrations by
Hugh Thomson

[Illustration: 1894]

Ruskin 156. Charing
House. Cross Road.

London
George Allen.

CHISWICK PRESS:--CHARLES WHITTINGHAM AND CO.
TOOKS COURT, CHANCERY LANE, LONDON.

[Illustration:

_To J. Comyns Carr
in acknowledgment of all I
owe to his friendship and
advice, these illustrations are
gratefully inscribed_

_Hugh Thomson_
]

PREFACE.

[Illustration]

_Walt Whitman has somewhere a fine and just distinction between “loving
by allowance” and “loving with personal love.” This distinction applies
to books as well as to men and women; and in the case of the not very
numerous authors who are the objects of the personal affection, it
brings a curious consequence with it. There is much more difference a